#  LangChain + 벡터 DB (FAISS) 기반으로 구성된 간단한 RAG 파이프라인 구현

흐름 : [문서 리스트] → [임베딩 & FAISS 저장] → [유사 문서 검색] → [GPT 응답 생성]

In [ ]:
# 1. 문서 정의
from langchain.docstore.document import Document

model_descriptions = [
    "GPT 모델은 OpenAI가 개발한 대규모 언어 모델 시리즈로서, GPT-4 등 다양한 버전을 통해 다국어 텍스트 생성과 대화형 AI를 비롯한 폭넓은 활용이 가능합니다.",
    "LLaMA 모델은 메타(Meta)에서 연구 목적으로 공개한 대규모 언어 모델로, 상대적으로 적은 자원으로도 고성능 텍스트 생성을 구현하도록 설계된 모델입니다.",
    "DeepSeek 모델은 최근 대규모 파라미터와 고도화된 언어 이해 능력을 갖추고 있다는 점에서 주목받고 있으며, 질의응답이나 문장 생성 등에서 우수한 성능을 보이는 모델로 알려져 있습니다.",
    "Gemma 모델은 첨단 NLP 기법을 적용해 다국어 텍스트 생성과 대화형 응용에 활용할 수 있도록 설계된 최신 대규모 언어 모델 중 하나입니다.",
    "KoGPT2 (SKT-AI) 모델은 GPT-2 구조를 한국어 데이터셋에 맞춰 학습하여 가벼운 파라미터 수 대비 자연스러운 한국어 텍스트 생성이 가능한 모델입니다.",
    "KoGPT6B (SKT-AI) 모델은 6B(60억) 파라미터를 갖춘 확장된 GPT-2 기반으로 대량의 한국어 텍스트를 학습해 긴 문맥 이해와 일관성 높은 문장 생성을 지원하는 모델입니다.",
    "KoBART (SKT-AI) 모델은 BART 구조를 한국어로 최적화해 요약, 번역, 생성 등 다양한 NLP 태스크에 활용할 수 있는 인코더-디코더 기반 모델입니다.",
    "KoT5 모델은 구글 T5를 한국어로 재학습하여 다방면의 텍스트 변환 및 생성 능력을 갖춘 범용 언어 모델입니다.",
    "KoAlpaca 모델은 LLaMA 기반을 한국어 인스트럭션 튜닝으로 최적화해 보다 자연스럽고 유창한 한국어 문장 생성을 목표로 설계된 모델입니다.",
    "Polyglot-Ko 시리즈 모델은 다중 언어 기반 학습 구조에서 한국어 최적화를 통해 텍스트 생성과 이해에 고른 성능을 보이는 모델입니다.",
    "mBART 모델은 페이스북 AI의 멀티링구얼 BART를 기반으로 25개 이상의 언어에서 번역, 요약 등 다양한 텍스트 생성에 활용 가능한 모델입니다.",
    "mT5 모델은 구글에서 공개한 다국어 T5로 100개 이상의 언어에서 질의응답, 번역, 요약 등 폭넓은 작업을 수행할 수 있는 범용 언어 모델입니다.",
    "BLOOM 모델은 BigScience 프로젝트에서 공개한 대규모 멀티링구얼 언어 모델로 40개 이상의 언어에서 자연어 이해와 생성을 지원하는 오픈소스 모델입니다."
]

documents = [Document(page_content=text) for text in model_descriptions]

In [18]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.2 MB/s eta 0:00:00


In [19]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [20]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.3 MB/s eta 0:00:00


In [ ]:
# 2. 임베딩 + FAISS 벡터스토어 구성
# 각 문서 → 벡터로 임베딩 → FAISS에 저장
# create_stuff_documents_chain() → 여러 문서를 단순히 “붙여서” GPT 입력으로 넘기는 방식
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embedding_model)

/tmp/ipython-input-8-1074053.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()


In [ ]:
# 3. 프롬프트 템플릿 + LLM 체인 정의
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7)

prompt_template = """
다음 문서 내용을 참고해서 질문에 간단히 답해주세요. 답변은 평문 텍스트로 작성해주세요.

문서 내용:
{context}

질문:
{input}
"""

stuff_prompt = PromptTemplate.from_template(prompt_template)
qa_chain = create_stuff_documents_chain(llm=llm, prompt=stuff_prompt)

In [ ]:
# 4. Retrieval + RAG 체인 구성
# retrieval : 벡터 기반 문서 검색기
# question_answer_chain : GPT가 context + 질문을 받아 답변 생성
# create_retrieval_chain : RAG 전체 파이프라인 구성
from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
rag_chain = create_retrieval_chain(retriever, qa_chain)

In [ ]:
# 5. 질문 실행
query = "상대적으로 적은 자원과 비용으로 활용할 수 있는 모델은 무엇이 있나요?"
result = rag_chain.invoke({"input": query})

In [ ]:
# 6. 결과 출력
print("\n답변:\n", result["answer"])



답변:
 LLaMA 모델입니다.


# MultiQueryRetriever를 활용한 LangChain RAG 코드

### MultiQueryRetriever
사용자 질문을 LLM을 통해 다양한 의미적 쿼리로 확장시킨 뒤, 이 여러 쿼리로 문서를 검색하고 병합하는 방식

#### 내부 작동원리:
1. 사용자 쿼리 → LLM을 통해 다양한 유사 질문들 생성 (예: 3~5개)
2. 각각을 벡터로 임베딩 → 벡터 DB에 검색
3. 여러 검색 결과를 통합 (중복 제거 포함)

#### 사용하기 적합한 상황
- 사용자가 모호한 질문을 던질 때
- 문서가 다양한 표현으로 구성되어 있을 때
- 기술적 키워드, 줄임말, 변형된 표현 등 포함 시
- 정확성이 중요한 문서 QA나 논문 분석

#### 주의할 것!
- MultiQueryRetriever는 LLM을 사용하므로 속도가 느릴 수 있고, 비용이 발생할 수 있습니다 (OpenAI, Claude 등 사용 시).

- 의미가 너무 확장되면 irrelevant document가 섞일 수 있습니다. → top_k 제한, 스코어 필터링으로 조절 필요

In [ ]:
# 1. 기본 설정 및 문서 준비
# 모델 설명을 리스트로 정의하고 LangChain 문서로 변환
from langchain.docstore.document import Document

model_descriptions = [
    "GPT 모델은 OpenAI가 개발한 대규모 언어 모델 시리즈로서, GPT-4 등 다양한 버전을 통해 다국어 텍스트 생성과 대화형 AI를 비롯한 폭넓은 활용이 가능합니다.",
    "LLaMA 모델은 메타(Meta)에서 연구 목적으로 공개한 대규모 언어 모델로, 상대적으로 적은 자원으로도 고성능 텍스트 생성을 구현하도록 설계된 모델입니다.",
    "DeepSeek 모델은 최근 대규모 파라미터와 고도화된 언어 이해 능력을 갖추고 있다는 점에서 주목받고 있으며, 질의응답이나 문장 생성 등에서 우수한 성능을 보이는 모델로 알려져 있습니다.",
    "Gemma 모델은 첨단 NLP 기법을 적용해 다국어 텍스트 생성과 대화형 응용에 활용할 수 있도록 설계된 최신 대규모 언어 모델 중 하나입니다.",
    "KoGPT2 (SKT-AI) 모델은 GPT-2 구조를 한국어 데이터셋에 맞춰 학습하여 가벼운 파라미터 수 대비 자연스러운 한국어 텍스트 생성이 가능한 모델입니다.",
    "KoGPT6B (SKT-AI) 모델은 6B(60억) 파라미터를 갖춘 확장된 GPT-2 기반으로 대량의 한국어 텍스트를 학습해 긴 문맥 이해와 일관성 높은 문장 생성을 지원하는 모델입니다.",
    "KoBART (SKT-AI) 모델은 BART 구조를 한국어로 최적화해 요약, 번역, 생성 등 다양한 NLP 태스크에 활용할 수 있는 인코더-디코더 기반 모델입니다.",
    "KoT5 모델은 구글 T5를 한국어로 재학습하여 다방면의 텍스트 변환 및 생성 능력을 갖춘 범용 언어 모델입니다.",
    "KoAlpaca 모델은 LLaMA 기반을 한국어 인스트럭션 튜닝으로 최적화해 보다 자연스럽고 유창한 한국어 문장 생성을 목표로 설계된 모델입니다.",
    "Polyglot-Ko 시리즈 모델은 다중 언어 기반 학습 구조에서 한국어 최적화를 통해 텍스트 생성과 이해에 고른 성능을 보이는 모델입니다.",
    "mBART 모델은 페이스북 AI의 멀티링구얼 BART를 기반으로 25개 이상의 언어에서 번역, 요약 등 다양한 텍스트 생성에 활용 가능한 모델입니다.",
    "mT5 모델은 구글에서 공개한 다국어 T5로 100개 이상의 언어에서 질의응답, 번역, 요약 등 폭넓은 작업을 수행할 수 있는 범용 언어 모델입니다.",
    "BLOOM 모델은 BigScience 프로젝트에서 공개한 대규모 멀티링구얼 언어 모델로 40개 이상의 언어에서 자연어 이해와 생성을 지원하는 오픈소스 모델입니다."
]

documents = [Document(page_content=text) for text in model_descriptions]

In [ ]:
# 2. 벡터스토어 (FAISS + OpenAI 임베딩) 구축
# 고속 벡터 검색을 위한 벡터 DB 구성
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embedding_model)

In [ ]:
# 3. LLM과 프롬프트 설정
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = ChatOpenAI(temperature=0.7)

prompt_template = """
다음 문서를 참고해서 질문에 간결하게 답해주세요. 제공된 문서에 없는 정보는 추론하지 마세요.

문서 내용:
{context}

질문:
{input}
"""

stuff_prompt = PromptTemplate.from_template(prompt_template)

# create_stuff_documents_chain
# 회수한 문서들을 LLM에게 전달해 응답 생성
qa_chain = create_stuff_documents_chain(llm=llm, prompt=stuff_prompt)

In [ ]:
# 4. LLM 기반 MultiQueryRetriever 사용
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import create_retrieval_chain

# LLM으로 질문을 다양한 표현으로 재작성하여 더 넓은 문서 검색 수행
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)

# 전체 Retrieval + QA 체인 구성
rag_chain = create_retrieval_chain(
    retriever=multi_query_retriever,
    combine_docs_chain=qa_chain
)

In [ ]:
# 5. 사용자 쿼리 실행
query = "LoRA를 활용했을 때 기존 방식과 비교해 어떤 이점이 있나요?"

result = rag_chain.invoke({"input": query})

In [ ]:
# 6. 결과 출력
print("\n답변:\n", result["answer"])


답변:
 LoRA를 활용했을 때 기존 방식과 비교해 전송 거리가 더 멀고 배터리 수명이 더 길며 더 적은 전력을 사용한다는 이점이 있습니다.


#  LangChain 기반 RAG + 텍스트 파일(txt)

#### 핵심 흐름 (RAG 파이프라인)
1. 텍스트 → 문서 청크로 나누기
2. 문서 청크 → 벡터로 임베딩
3. FAISS 벡터 DB에 저장
4. 질문 → 관련 문서 검색
5. 관련 문서 + 질문 → LLM 답변 생성

In [ ]:
# 1. 텍스트 분할기 및 데이터 로딩
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

file_path = '/content/drive/MyDrive/Projects/LLM/RAG/lora.txt'
with open(file_path, encoding='utf-8') as f:
    all_text = f.read()

# RecursiveCharacterTextSplitter
# : 기본적으로 문장 부호나 단락을 기준으로 최대한 의미 단위로 쪼개려고 함.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,          # 각 청크 최대 200자
    chunk_overlap=20,        # 청크 간 20자 중복 | overlap을 줘서 문맥 단절을 방지
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 구분자 단순 문자로 인식
)

# 청크 분할 수행
# create_documents()
# : 각 청크를 Document 객체로 생성
texts = text_splitter.create_documents([all_text])
documents = [Document(page_content=text.page_content) for text in texts]


In [ ]:
# 2. 임베딩 모델 및 벡터스토어 생성(FAISS)
# FAISS
# : Facebook에서 만든 빠른 유사도 검색 라이브러리
# 각 문서를 벡터로 변환 후, FAISS 벡터 인덱스에 저장
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embedding_model)

In [ ]:
# 3. Retrieval-augmented QA 체인 구성
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7)

stuff_prompt = PromptTemplate.from_template("""
다음 문서 내용을 참고하여 질문에 대해 3문장 이내로 간략히 답을 제시해주세요. 답변에는 평문 텍스트를 사용해주세요.

문서내용:
{context}

질문: {input}

답변:
""")

qa_chain = create_stuff_documents_chain(llm=llm, prompt=stuff_prompt)
retriever = vectorstore.as_retriever()
chain = create_retrieval_chain(retriever, qa_chain)

In [ ]:
# 4. 질문 예시 및 결과 출력
query_1 = "lora를 활용했을 때, 기존 방식에 비해서 어떤 이점이 있는지 설명해주세요"
query_2 = "상대적으로 적은 자원과 비용으로 활용할 수 있는 모델은 뭐가 있나요?"

for query in [query_1, query_2]:
    result = chain.invoke({'input': query})
    print(f"\n[질문] {query}\n[답변] {result['answer']}")



[질문] lora를 활용했을 때, 기존 방식에 비해서 어떤 이점이 있는지 설명해주세요
[답변] LORA를 활용하면 더 빠르고 유연한 실험 환경을 제공하여 적은 리소스로도 다양한 미세 조정 전략을 시도할 수 있습니다. 또한, 파라미터 수를 줄이면서도 성능을 유지하거나 향상시킬 수 있어 더 효율적인 결과를 얻을 수 있습니다. 미세 조정 과정에서 학습 속도가 향상되어 계산 비용을 줄이고 더 많은 실험을 수행할 수 있습니다.

[질문] 상대적으로 적은 자원과 비용으로 활용할 수 있는 모델은 뭐가 있나요?
[답변] LORA (Low-Rank Adaptation) 모델은 상대적으로 적은 자원과 비용으로 활용할 수 있는 모델입니다.


In [ ]:
# 5. MMR(Maximal Marginal Relevance) 기반 검색으로 변경하여 재질문

# 왜?
# - 중복되지 않으면서도 관련 있는 문서들을 선택함
#   → 덕분에 더 풍부하고 정확한 문맥을 LLM에 전달 가능
# - 질의의 의도와 문맥을 더 잘 반영하기 위해

retriever_mmr = vectorstore.as_retriever(search_type='mmr')
chain_mmr = create_retrieval_chain(retriever_mmr, qa_chain)

result_mmr = chain_mmr.invoke({'input': query_2})
print(f"\n[MMR 기반 답변] {result_mmr['answer']}")


[MMR 기반 답변] 대규모 LLM을 미세 조정할 수 있는 LORA 모델이 있습니다.


# LangChain + HuggingFace 기반 RAG (Retrieval-Augmented Generation) 구조 구축

- Gemma LLM을 직접 로딩하여 응답 생성에 사용

- HuggingFace Embedding 모델을 직접 래핑해서 커스텀 벡터 임베딩

- FAISS 기반 벡터 검색기 구축

- MMR (Max Marginal Relevance)을 이용한 리트리버 실험 포함

- <흐름> : [질문] → [벡터 임베딩] → [관련 문서 검색] → [문서 + 질문 → 프롬프트] → [Gemma 모델 응답 생성]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.embeddings.base import Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

In [ ]:
# 1. LLM (Gemma) 로딩
model_id = "google/gemma-3-1b-it"
model = AutoModelForCausalLM.from_pretrained(model_id).to('cuda').eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)

stop_token_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<end_of_turn>")]

gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=3000,
    temperature=0.1,
    do_sample=False,
    eos_token_id=stop_token_ids
)

llm = HuggingFacePipeline(pipeline=gen_pipeline)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-26-57556909.py:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U 

In [ ]:
# 2. 커스텀 임베딩 클래스 정의
class CustomEmbeddings(Embeddings):
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device

    def _mean_pooling(self, hidden_states, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
        sum_embeddings = torch.sum(hidden_states * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        return sum_embeddings / sum_mask

    def embed_documents(self, texts):
        embeddings = []
        for text in texts:
            with torch.no_grad():
                inputs = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt').to(self.device)
                outputs = self.model(**inputs, output_hidden_states=True, return_dict=True)
                pooled = self._mean_pooling(outputs.hidden_states[-1], inputs['attention_mask'])
                embeddings.append(pooled[0].cpu().tolist())
        return embeddings

    def embed_query(self, text):
        return self.embed_documents([text])[0]

embedding_model = CustomEmbeddings(model, tokenizer, device='cuda')

In [ ]:
# 3. 문서 정의 및 벡터스토어 생성
model_descriptions = [
    "GPT 모델은 OpenAI가 개발한 대규모 언어 모델 시리즈로서, GPT-4 등 다양한 버전을 통해 다국어 텍스트 생성과 대화형 AI를 비롯한 폭넓은 활용이 가능합니다.",
    "LLaMA 모델은 메타(Meta)에서 공개한 대규모 언어 모델로, 적은 자원으로 고성능 텍스트 생성을 구현할 수 있도록 설계된 모델입니다.",
    "DeepSeek 모델은 고도화된 언어 이해 능력을 갖춘 대규모 모델로, 질의응답이나 문장 생성 등에서 우수한 성능을 보입니다.",
    "Gemma 모델은 첨단 NLP 기법을 적용한 다국어 생성 및 대화용 최신 언어 모델입니다.",
    "KoGPT2 모델은 GPT-2 구조를 한국어 데이터셋으로 학습하여 가볍고 자연스러운 한국어 생성이 가능합니다.",
    "KoGPT6B 모델은 6B 파라미터로 긴 문맥 이해와 고성능 한국어 생성이 가능한 모델입니다.",
    "KoBART 모델은 요약·번역 등 다양한 작업에 활용 가능한 한국어 특화 BART 모델입니다."
]

documents = [Document(page_content=text) for text in model_descriptions]
vectorstore = FAISS.from_documents(documents, embedding_model)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# 4. 프롬프트 템플릿 및 QA 체인 정의
template = """
다음 문서 내용을 참고하여 질문에 대해 3문장 이내로 간단히 답해주세요. 답변에는 평문 텍스트를 사용해주세요.

문서내용:
{context}

질문:
{input}

답변:
"""

stuff_prompt = PromptTemplate(template=template)
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=stuff_prompt)
retriever = vectorstore.as_retriever()
chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# 5. 질문 및 결과 출력
queries = [
    "한국어 생성이 가능한 모델은 어떤 것들이 있는지 알려줘",
    "상대적으로 적은 자원과 비용으로 활용할 수 있는 모델은 뭐가 있나요?"
]

for q in queries:
    result = chain.invoke({'input': q})
    print(f"질문: {q}\n답변: {result['answer']}\n")


`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True}. If this is not desired, please set these values explicitly.


질문: 한국어 생성이 가능한 모델은 어떤 것들이 있는지 알려줘
답변: 
다음 문서 내용을 참고하여 질문에 대해 3문장 이내로 간단히 답해주세요. 답변에는 평문 텍스트를 사용해주세요.

문서내용:
DeepSeek 모델은 고도화된 언어 이해 능력을 갖춘 대규모 모델로, 질의응답이나 문장 생성 등에서 우수한 성능을 보입니다.

LLaMA 모델은 메타(Meta)에서 공개한 대규모 언어 모델로, 적은 자원으로 고성능 텍스트 생성을 구현할 수 있도록 설계된 모델입니다.

KoBART 모델은 요약·번역 등 다양한 작업에 활용 가능한 한국어 특화 BART 모델입니다.

KoGPT2 모델은 GPT-2 구조를 한국어 데이터셋으로 학습하여 가볍고 자연스러운 한국어 생성이 가능합니다.

질문:
한국어 생성이 가능한 모델은 어떤 것들이 있는지 알려줘

답변:
KoGPT2 모델, KoBART 모델, KoLL 모델입니다.


질문: 상대적으로 적은 자원과 비용으로 활용할 수 있는 모델은 뭐가 있나요?
답변: 
다음 문서 내용을 참고하여 질문에 대해 3문장 이내로 간단히 답해주세요. 답변에는 평문 텍스트를 사용해주세요.

문서내용:
DeepSeek 모델은 고도화된 언어 이해 능력을 갖춘 대규모 모델로, 질의응답이나 문장 생성 등에서 우수한 성능을 보입니다.

LLaMA 모델은 메타(Meta)에서 공개한 대규모 언어 모델로, 적은 자원으로 고성능 텍스트 생성을 구현할 수 있도록 설계된 모델입니다.

KoBART 모델은 요약·번역 등 다양한 작업에 활용 가능한 한국어 특화 BART 모델입니다.

KoGPT2 모델은 GPT-2 구조를 한국어 데이터셋으로 학습하여 가볍고 자연스러운 한국어 생성이 가능합니다.

질문:
상대적으로 적은 자원과 비용으로 활용할 수 있는 모델은 뭐가 있나요?

답변:
KoBART 모델, KoGPT2 모델, LLaMA 모델입니다.




In [ ]:
# 6. MMR retriever 실험
retriever_mmr = vectorstore.as_retriever(search_type='mmr')
mmr_chain = create_retrieval_chain(retriever_mmr, question_answer_chain)
result = mmr_chain.invoke({'input': queries[1]})
print(f"MMR 기반 리트리버 응답:\n{result['answer']}")

MMR 기반 리트리버 응답:

다음 문서 내용을 참고하여 질문에 대해 3문장 이내로 간단히 답해주세요. 답변에는 평문 텍스트를 사용해주세요.

문서내용:
DeepSeek 모델은 고도화된 언어 이해 능력을 갖춘 대규모 모델로, 질의응답이나 문장 생성 등에서 우수한 성능을 보입니다.

KoGPT2 모델은 GPT-2 구조를 한국어 데이터셋으로 학습하여 가볍고 자연스러운 한국어 생성이 가능합니다.

LLaMA 모델은 메타(Meta)에서 공개한 대규모 언어 모델로, 적은 자원으로 고성능 텍스트 생성을 구현할 수 있도록 설계된 모델입니다.

KoBART 모델은 요약·번역 등 다양한 작업에 활용 가능한 한국어 특화 BART 모델입니다.

질문:
상대적으로 적은 자원과 비용으로 활용할 수 있는 모델은 뭐가 있나요?

답변:
KoGPT2 모델, LLaMA 모델, KoBART 모델입니다.



# 문서 임베딩 후 유사도 기반 검색

In [3]:
from sentence_transformers import SentenceTransformer    # 문장 수준 의미 임베딩
from sklearn.metrics.pairwise import cosine_similarity   # 가장 의미 가까운 문서 검색

In [4]:
documents = [
    "사과는 빨갛고 맛있는 과일입니다. 비타민 C가 풍부하여 건강에 좋습니다.",
    "서울은 대한민국의 수도입니다. 한강이 도시를 가로지르며 아름다운 야경을 선사합니다.",
    "기린은 목이 긴 동물로 아프리카 초원에 서식합니다. 주로 나뭇잎을 먹습니다.",
    "커피는 전 세계적으로 인기 있는 음료입니다. 다양한 맛과 향이 있습니다.",
    "독서는 지식을 넓히고 상상력을 자극하는 훌륭한 취미입니다. 마음의 양식이라고도 불립니다."
]

In [5]:
# SentenceTransformer로 문서 임베딩
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')
document_embeddings = model_embedding.encode(documents)
document_embeddings.shape

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(5, 384)

document_embeddings.shape
- (5, 384) = (문서 개수, 문서 하나 당 갖는 차원 수)

In [6]:
# 질의와 문서 간 유사도 계산
query = "아프리카 기린은 무엇을 먹나요."
query_embedding = model_embedding.encode(query)
similarities = cosine_similarity([query_embedding], document_embeddings)

In [7]:
# 가장 유사한 문서 선택
most_similar_doc_index = similarities.argmax()
retrieved_doc = documents[most_similar_doc_index]
retrieved_doc

'기린은 목이 긴 동물로 아프리카 초원에 서식합니다. 주로 나뭇잎을 먹습니다.'

# 선택된 문서와 질문을 기반으로 LLM 응답 생성 (Gemma 사용)

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch

In [11]:
model_id = 'google/gemma-3-1b-it'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)
# model = model.to("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [12]:
# 질의 프롬프트 생성
# prompt = f"정보: ... 질문: ..." 형태는 instruct 기반 모델에서 권장
context = retrieved_doc
prompt = f"다음 정보를 바탕으로 질문에 답해주세요:\n정보: {context}\n질문: {query}\n답변:"

In [13]:
# 토큰화 및 생성
input_ids = tokenizer(prompt, return_tensors='pt').to(model.device)

In [14]:
# 출력 제어
# 줄바꿈으로 멈추는 기준 정의
# Stopping Criteria
# : eos_token_id로도 가능하지만, 더 정교하게 통제하고 싶을 때 유용
# : API inference시에도 응답 길이 통제를 위해 자주 사용됨
class StopOnNewline(StoppingCriteria):
    def __call__(self, input_ids, scores):
        return input_ids[0, -1].item() == tokenizer.convert_tokens_to_ids("\n")

stop_criteria = StoppingCriteriaList([StopOnNewline()])

In [15]:
# 응답 생성
outputs = model.generate(
    **input_ids,
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id,
    stopping_criteria=stop_criteria
)

#### ▶ generate() 옵션 정리
| 옵션 이름                | 설명                                  |
| -------------------- | ----------------------------------- |
| `input_ids`          | 입력 시퀀스                              |
| `max_new_tokens`     | 새로 생성할 최대 토큰 수                      |
| `temperature`        | 생성 분포의 무작위성 조절 (0\~1: 보수적, >1: 창의적) |
| `top_p`              | 누적 확률 기반의 nucleus sampling (0.9 추천) |
| `repetition_penalty` | 반복 방지 계수 (1.1\~1.2 추천)              |
| `eos_token_id`       | 종료 토큰 지정                            |
| `stopping_criteria`  | 사용자 정의 중단 조건 (고급 사용)                |


In [16]:
# 응답 디코딩
input_len = input_ids['input_ids'].size(1)
answer = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)
print("답변:", answer)

답변:  기린은 주로 나뭇잎을 먹습니다.

정보: 해바라기는 잎을 가지고 있고, 햇빛을 흡수하여 생명을 유지합니다.



# 이메일 요약 파이프라인 (LangChain + Pydantic Parser)

In [21]:
from langchain.output_parsers import PydanticOutputParser    # PydanticOutputParser를 이용하면 LLM 출력 포맷 강제 가능
from langchain_core.prompts import PromptTemplate    # 프롬프트 구조화
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [22]:
# JSON 구조 정의
class EmailSummary(BaseModel):
    person: str = Field(..., description='발신자 이름')
    email: str = Field(..., description='발신자 이메일')
    subject: str = Field(..., description='메일 제목')
    summary: str = Field(..., description='메일 내용 요약')
    date: str = Field(..., description='미팅 제안 날짜 및 시간')

In [23]:
# 파서 및 LLM 설정
parser = PydanticOutputParser(pydantic_object=EmailSummary)
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

In [24]:
# 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    template="""
메일 내용을 분석하여 다음 JSON 형식에 맞춰 요약해 주세요.
미팅 제안 날짜와 시간이 있다면 해당 정보를 'date' 필드에 정확히 기입하고, 없다면 빈 문자열로 남겨주세요.

{format_instructions}

메일 내용:
{email_content}
""",
    input_variables=["email_content"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [25]:
# 체인 구성
email_chain = prompt_template | llm | parser


In [26]:
# 테스트 이메일
email_example = """
보낸 사람: 김영희 <younghee.kim@example.com>
제목: 다음 주 미팅 관련 문의
내용:
안녕하세요, 김철수님.
지난 번 논의드렸던 프로젝트 관련하여 다음 주 중으로 미팅을 가질 수 있을지 문의드립니다.
가능하시다면 7월 23일 오전 10시나 7월 24일 오후 2시 중으로 시간을 제안해주시면 감사하겠습니다.
미리 의제는 공유드리겠습니다.
감사합니다.
"""

In [27]:
# 요약 실행
parsed_summary = email_chain.invoke({"email_content": email_example})
print(parsed_summary.dict())

{'person': '김영희', 'email': 'younghee.kim@example.com', 'subject': '다음 주 미팅 관련 문의', 'summary': '지난 번 논의드렸던 프로젝트 관련하여 다음 주 중으로 미팅을 가질 수 있는지 문의', 'date': '7월 23일 오전 10시 또는 7월 24일 오후 2시'}


/tmp/ipython-input-27-2135934941.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(parsed_summary.dict())


# 다양한 데이터 로더 예시 (LangChain 커뮤니티 제공)

- PDF: PyPDF / PyMuPDF (텍스트 추출 품질 차이 존재)  →  PyMuPDF가 PyPDF보다 레이아웃 보존이 더 뛰어남

- Web: bs4.SoupStrainer로 타겟 영역만 추출  →  WebBaseLoader: 웹 스크래핑 시 유용한 유연한 구조

- ArXiv: 메타 정보 포함 논문 로딩  →  자동 논문 검색 및 분석에 활용

## PDF 로더: PyPDFLoader or PyMuPDFLoader

In [31]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 59.3 MB/s eta 0:00:00


In [28]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

In [32]:
pdf_path = '/content/drive/MyDrive/Projects/LLM/RAG/b2b_technical_solution.pdf'
loader = PyMuPDFLoader(pdf_path)  # 또는 PyPDFLoader
pdf_pages = loader.load()
print("첫 페이지 내용:", pdf_pages[0].page_content)

첫 페이지 내용: 📄 InsightIQ B2B 솔루션 기술 자료 
문서 버전: 1.0 최종 수정일: 2025 년 7 월 9 일 
 
## 1. 제품 소개서: AI-Powered Analytics Dashboard "InsightIQ" 
1.1 제품 개요 
InsightIQ 는 기업의 판매 데이터를 자동으로 분석하여 마케팅 전략 수립에 
필요한 인사이트를 제공하는 B2B SaaS 솔루션입니다. 실시간 대시보드와 자동 
리포팅 기능을 통해 데이터 기반 의사결정을 지원합니다. 
1.2 주요 기능 
• 
실시간 데이터 대시보드: 매출, 고객 유입 경로, 전환율 등 핵심 지표를 
실시간으로 시각화합니다. 
• 
AI 기반 이상 감지: 데이터 패턴을 학습하여 평소와 다른 이상 신호(매출 
급감, 특정 제품 이탈률 증가 등)를 자동으로 감지하고 알림을 보냅니다. 
• 
자동 주간 리포트: 매주 월요일 오전 9 시, 핵심 성과 지표(KPI)와 주요 
변동 사항을 요약한 리포트를 이메일로 자동 발송합니다. 
1.3 요금제 
• 
Basic Plan: 월 10 만원, 사용자 3 명, 대시보드 5 개 
• 
Pro Plan: 월 30 만원, 사용자 10 명, 대시보드 무제한, AI 이상 감지 기능 
포함 
• 
Enterprise Plan: 별도 문의, Pro Plan 기능 전체 포함, 전담 매니저 및 
기술 지원 
1.4 기술 요구사항 
• 
데이터 연동: REST API 또는 CSV 파일 업로드를 통해 데이터 연동이 
가능합니다. 
• 
지원 브라우저: Chrome, Safari, Firefox 최신 버전 
 
## 2. 기술 지원: InsightIQ API 가이드 FAQ 
Q: API 인증은 어떻게 하나요?


## 웹 페이지 로더

In [33]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

In [34]:
web_loader = WebBaseLoader(
    web_paths=["https://n.news.naver.com/article/437/0000378416"],
    bs_kwargs=dict(parse_only=bs4.SoupStrainer("div", attrs={"class": ["newsct_article _article_body", "media_end_head_title"]})),
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
    }
)
web_docs = web_loader.load()
print("웹 문서 개수:", len(web_docs))

웹 문서 개수: 1


## ArXiv 논문 로더

In [37]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=3466144cb10fc0555674df0dfbc524a1f3231c424d6cdbb52d01743b36255e87
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [38]:
from langchain_community.document_loaders import ArxivLoader

In [39]:
arxiv_loader = ArxivLoader(query="large language model", load_max_docs=3, load_all_available_meta=True)
arxiv_docs = arxiv_loader.load()
print("ArXiv 문서 수:", len(arxiv_docs))

ArXiv 문서 수: 3
